In [ ]:
#!python
import logging
from kiteconnect import KiteConnect, KiteTicker, exceptions
import pandas as pd
import numpy as np
import datetime as dt
import sys
import subprocess
import pkg_resources
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time, pyotp
from helper_functions import *
import json
from glob import glob
import os
import time

# required = {'pydrive'}
# install_package(required)

In [ ]:
request_times = [0,0]

In [ ]:
def get_connection_info(file):
    try:
        with open('connection_info.json') as json_file:
            data = json.load(json_file)
    except:
        print('Error in loading data')
        return
    return data

def historical_data(symbol, from_date, to_date, interval):
    """
    This function will return historical data of the instrument for specific period of days for specific interval
    """
    
    df = pd.DataFrame()   
    int_token = instrument_token(master_df, symbol)  #the function we defined above which will return token no. of instrument
    
    to_date   = pd.Timestamp(to_date)
    from_date = pd.Timestamp(from_date)
    
    limit_dict = {'minute' : 60, '3minute' : 100, '5minute' : 100, '10minute' : 100, '15minute' : 200, '30minute' : 200, '60minute' : 400, 'day' : 2000}
    limit = limit_dict[interval]
    while True:
        if from_date >= (to_date - dt.timedelta(limit)):
            #if from_date is within the 60 days limit
            while time.time() - request_times[-2] < 1:
                pass
            temp_df = pd.DataFrame(kite.historical_data(int_token, from_date, to_date, interval))
            request_times.append(time.time())
            df = pd.concat([df, temp_df])
            break
            
        else:                                                            #if from_date has more than 60 days limit
            to_date_new = from_date + dt.timedelta(limit)
            while time.time() - request_times[-2] < 1:
                pass
            temp_df = pd.DataFrame(kite.historical_data(int_token, from_date, to_date_new, interval))
            request_times.append(time.time())
            df = pd.concat([df, temp_df])
            #to_date = from_date.date() + dt.timedelta(60)
            from_date = to_date_new
            
    return df

In [ ]:
os.chdir('/Users/yash/Desktop/Trading/kite connect/code')

In [ ]:
connection_file = 'connection_info.json'
conInfo = get_connection_info(connection_file)

kite = login_in_zerodha(conInfo['KITE_API_KEY'],
                        conInfo['KITE_API_SECRET'],
                        conInfo['USER'],
                        conInfo['PASSWORD'],
                        conInfo['TOTP_KEY'])
print(kite.profile())

In [ ]:
nse_df = pd.DataFrame(kite.instruments('NSE'))
bse_df = pd.DataFrame(kite.instruments('BSE'))
master_df = pd.concat([nse_df, bse_df])
master_df.drop_duplicates(subset="name",
                     keep='first', inplace=True)

In [ ]:
start_ = '2018-01-01'
end_ = '2022-12-31'

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
from glob import glob

gauth = GoogleAuth()           
drive = GoogleDrive(gauth) 

In [ ]:
path = '/Users/yash/Desktop/Trading/kite connect/historical_data/minute/'

In [ ]:
os.chdir(path)

In [ ]:
len(master_df)

In [ ]:
#list
done_set = set()
minute_folder = '1tc6yb3kkqmgyDh7am7SU17_FupTlBg9o'
drive_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(minute_folder)}).GetList()

for file in drive_list:
#     print(file['title'])
#     break
    done_set.add(file['title'].split('/')[-1].split('_')[0])
print(len(done_set))
files = glob(path + '*')

for file in files:
#     print(file['title'])
#     break
    done_set.add(file.split('/')[-1].split('_')[0])
print(len(done_set))

In [ ]:
for i, j in master_df.iterrows():
    try:
        symbol = j.tradingsymbol
        fname = symbol + '_' + start_.replace('-','') + '_' + end_.replace('-','')
        if symbol in done_set:
    #         done_set.add(symbol)a
            print(i, 'Skipping ', symbol)
            continue
        df = historical_data(symbol, start_ , end_, "minute")
        df.to_csv(fname)
        done_set.add(symbol)

        files = glob('/Users/yash/Desktop/Trading/kite connect/historical_data/minute/*')
        while len(files) > 500:
            print('Waiting')
            pass
        print(i, j.tradingsymbol)
    except:
        pass